# Adding paths and files

In [13]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..', '..'))
import config,utils

# Specify the directory where your CSV files are located
directory = os.path.join(config.RAW_DATA_DIR, 't20s_csv2')
print(directory)
from hdfs import InsecureClient
client = InsecureClient(config.HDFS_URL, user=config.HDFS_USER)

all_files = client.list(directory)
info_files = [os.path.join(directory, file) for file in all_files if 'info' in file]

matches=[]
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split('/')[-1])

match_ids=[]
for csv_file in matches:
    match_ids.append(csv_file.split('_')[0])

/usr/ravi/t20/data/1_rawData/t20s_csv2
[2024-11-19T00:52:54.399+0530] {client.py:192} INFO - Instantiated <InsecureClient(url='http://192.168.245.142:9870')>.
[2024-11-19T00:52:54.400+0530] {client.py:1116} INFO - Listing '/usr/ravi/t20/data/1_rawData/t20s_csv2'.


In [16]:
import pandas as pd
import io
def process_single_match(client, match_id: str) -> pd.DataFrame:
    """
    Process a single match file and return the processed DataFrame or None on failure.

    Args:
        client: The HDFS client for file operations.
        match_id: Unique ID for each match.

    Returns:
        DataFrame of the processed match or None if an error occurs.
    """
    try:
        with client.read(os.path.join(config.RAW_DATA_DIR, 't20s_csv2', f'{match_id}_info.csv')) as reader:
            data = reader.read()
        match_df = pd.read_csv(io.StringIO(data.decode('utf-8')), header=None, names=['col1', 'attributes', 'values', 'players', 'code'])
        match_df = match_df.drop(columns=['col1', 'players', 'code']).T
        match_df.columns = match_df.iloc[0]
        match_df['match_id'] = match_id
        match_df = match_df[['match_id', 'team', 'team', 'gender', 'season', 'winner']].drop('attributes')
        match_df = match_df.reset_index(drop=True)
        return match_df
    except Exception as e:
        print(f"Error processing match {match_id}: {e}")
        return None

In [20]:
from tqdm import tqdm
import concurrent.futures
import logging
logging.basicConfig(level=logging.ERROR)
recalculated_matches = []

# Set the logging level for the HDFS client to WARNING
logging.getLogger('hdfs.client').setLevel(logging.CRITICAL)
injured_matches = []

# Process each match concurrently using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {
        executor.submit(process_single_match, client, match_id): match_id
        for match_id in match_ids
    }

    # Gather results with tqdm for progress tracking
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing Matches"):
        match = futures[future]
        try:
            result = future.result()
            if result is not None:
                recalculated_matches.append(result)
            else:
                injured_matches.append(match)
        except Exception as e:
            print(f"Error in future for match {match}: {e}")
            injured_matches.append(match)

if not recalculated_matches:
    print('No matches were successfully processed.')
    raise ValueError('No matches were successfully processed.')

matches_data = pd.concat(recalculated_matches, ignore_index=True)
matches_data

Processing Matches:   1%|          | 46/3800 [00:00<00:18, 207.31it/s]

Error processing match 1041617: "['winner'] not in index"
Error processing match 1115799: "['winner'] not in index"


Processing Matches:   3%|▎         | 106/3800 [00:00<00:20, 179.52it/s]

Error processing match 1123209: "['winner'] not in index"


Processing Matches:   4%|▍         | 162/3800 [00:00<00:22, 163.79it/s]

...

Error processing match 730293: "['winner'] not in index"


Processing Matches:  98%|█████████▊| 3714/3800 [00:21<00:00, 156.45it/s]

Error processing match 902653: "['winner'] not in index"
Error processing match 951319: "['winner'] not in index"


Processing Matches: 100%|██████████| 3800/3800 [00:21<00:00, 175.87it/s]


attributes,match_id,team,team,team,team,gender,season,winner
0,1001349,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Sri Lanka
1,1007657,Zimbabwe,India,Zimbabwe,India,male,2016,India
2,1001351,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Sri Lanka
3,1001353,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Australia
4,1004729,Ireland,Hong Kong,Ireland,Hong Kong,male,2016,Hong Kong
...,...,...,...,...,...,...,...,...
3682,966739,Hong Kong,United Arab Emirates,Hong Kong,United Arab Emirates,male,2015/16,United Arab Emirates
3683,966753,Bangladesh,Sri Lanka,Bangladesh,Sri Lanka,male,2015/16,Bangladesh
3684,966763,Pakistan,Sri Lanka,Pakistan,Sri Lanka,male,2015/16,Pakistan
3685,966765,Bangladesh,India,Bangladesh,India,male,2015/16,India


In [21]:
if recalculated_matches:
    # Concatenate all match DataFrames
    matches_data = pd.concat(recalculated_matches, ignore_index=True)
    matches_data.columns = ['match_id', 'team1', 'team2', 'team1_duplicate', 'team2_duplicate', 'gender', 'season', 'winner']
    matches_data = matches_data.drop(columns=['team1_duplicate', 'team2_duplicate'])

    # Data quality checks
    if matches_data.empty:
        print("No match data consolidated.")
        raise ValueError("Consolidated match data is empty.")
    required_columns = ["match_id", "team1", "team2", "gender", "season", "winner"]
    missing_columns = [col for col in required_columns if col not in matches_data.columns]
    if missing_columns:
        print(f"Missing columns in matches data: {missing_columns}")
        raise ValueError(f"Missing columns in matches data: {missing_columns}")
    
    # Save matches_data directly to HDFS
    utils.ensure_hdfs_directory(client, config.PROCESSED_DATA_DIR)
    matches_csv_path = os.path.join(config.PROCESSED_DATA_DIR, 'matches.csv')
    csv_data = matches_data.to_csv(index=False)
    client.write(matches_csv_path, data=csv_data, overwrite=True)
    print('Matches data processing and saving completed successfully.')
else:
    print('No matches were successfully processed.')
    raise Exception('No matches were successfully processed.')

print(f'Successfully processed matches: {len(recalculated_matches)}')
print(f'Failed matches: {len(injured_matches)}')

Matches data processing and saving completed successfully.
Successfully processed matches: 3687
Failed matches: 113
